<a href="https://colab.research.google.com/github/RaihanHeggi/markov_chain_ddos_attack/blob/main/classification_testing_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sn 
import numpy as np 
import tensorflow as tf
from tensorflow.keras import datasets, layers, models 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

## **Loading and Processing Data**

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/RaihanHeggi/markov_chain_ddos_attack/main/semi_processed_train.csv', sep=',')
df_2 = pd.read_csv('https://raw.githubusercontent.com/RaihanHeggi/markov_chain_ddos_attack/main/dataset_sdn.csv', sep=';')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head(1)

,dt,switch,pktcount,bytecount,dur,dur_nsec,flows,packetins,pktrate,Pairflow,Protocol,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps,label
0,0.220956,0.0,0.174242,0.328245,0.053163,0.716717,0.053723,0.066667,0.076883,0.962331,0.996744,0.96243,0.0,1.0,0.5,0.113329,0.000003,0.0


In [3]:
#Changing Protocol

df['Protocol'] = df_2['Protocol'] 
df['Protocol'] = df['Protocol'].replace('UDP', 1)
df['Protocol'] = df['Protocol'].replace('TCP', 2)
df['Protocol'] = df['Protocol'].replace('ICMP', 3)

In [14]:
df.shape

(94797, 18)

### **Split data to X and Y**

In [15]:
x = df[df.columns[:17]]
y = df['label']

In [16]:
x.head(1)

,dt,switch,pktcount,bytecount,dur,dur_nsec,flows,packetins,pktrate,Pairflow,Protocol,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps
0,0.220956,0.0,0.174242,0.328245,0.053163,0.716717,0.053723,0.066667,0.076883,0.962331,1,0.96243,0.0,1.0,0.5,0.113329,0.000003


In [19]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=6)

## **Creating Artificial Neural Network Model**

In [22]:
model = models.Sequential([
    layers.Dense(2000, input_dim=17, activation='relu'),
    layers.Dense(500, activation='sigmoid'),
    layers.Dense(500, activation='relu'),
    layers.Dense(2, activation='softmax')
])
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
2074/2074 [==============================] - 42s 20ms/step - loss: 0.2257 - accuracy: 0.8950
Epoch 2/5
2074/2074 [==============================] - 40s 19ms/step - loss: 0.1298 - accuracy: 0.9418
Epoch 3/5
2074/2074 [==============================] - 39s 19ms/step - loss: 0.1079 - accuracy: 0.9495
Epoch 4/5
2074/2074 [==============================] - 38s 19ms/step - loss: 0.0994 - accuracy: 0.9544
Epoch 5/5
2074/2074 [==============================] - 39s 19ms/step - loss: 0.0903 - accuracy: 0.9583


### **Predict Classification**

In [23]:
y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)

### **Training**

In [24]:
import warnings
warnings.filterwarnings('ignore')
y_pred_train = [np.argmax(element) for element in y_pred_train]
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))
print("Accuracy:",accuracy_score(y_train, y_pred_train)*100)


[[39362  2081]
 [  453 24461]]
              precision    recall  f1-score   support

         0.0       0.99      0.95      0.97     41443
         1.0       0.92      0.98      0.95     24914

    accuracy                           0.96     66357
   macro avg       0.96      0.97      0.96     66357
weighted avg       0.96      0.96      0.96     66357

Accuracy: 96.18126196181261


### **Testing**

In [25]:
warnings.filterwarnings('ignore')
y_pred_test = [np.argmax(element) for element in y_pred_test]
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))
print("Accuracy:",accuracy_score(y_test, y_pred_test)*100)

[[16935   913]
 [  193 10399]]
              precision    recall  f1-score   support

         0.0       0.99      0.95      0.97     17848
         1.0       0.92      0.98      0.95     10592

    accuracy                           0.96     28440
   macro avg       0.95      0.97      0.96     28440
weighted avg       0.96      0.96      0.96     28440

Accuracy: 96.11111111111111


### **Validation**

In [31]:
df_val = pd.read_csv('https://raw.githubusercontent.com/RaihanHeggi/markov_chain_ddos_attack/main/semi_processed_train.csv', sep=',')
df_val = df_val.loc[:, ~df_val.columns.str.contains('^Unnamed')]
df_val['Protocol'] = df_2['Protocol'] 
df_val['Protocol'] = df_val['Protocol'].replace('UDP', 1)
df_val['Protocol'] = df_val['Protocol'].replace('TCP', 2)
df_val['Protocol'] = df_val['Protocol'].replace('ICMP', 3)
df_val_x = df_val[df_val.columns[:17]]
df_val_y = df_val['label']
df_val_x.head(1)

,dt,switch,pktcount,bytecount,dur,dur_nsec,flows,packetins,pktrate,Pairflow,Protocol,port_no,tx_bytes,rx_bytes,tx_kbps,rx_kbps,tot_kbps
0,0.220956,0.0,0.174242,0.328245,0.053163,0.716717,0.053723,0.066667,0.076883,0.962331,1,0.96243,0.0,1.0,0.5,0.113329,0.000003


In [33]:
y_val_pred = model.predict(df_val_x)

In [36]:
warnings.filterwarnings('ignore')
y_val_pred = [np.argmax(element) for element in y_val_pred]
print(confusion_matrix(df_val_y, y_val_pred))
print(classification_report(df_val_y,y_val_pred))
print("Accuracy:",accuracy_score(df_val_y, y_val_pred)*100)

[[56297  2994]
 [  646 34860]]
              precision    recall  f1-score   support

         0.0       0.99      0.95      0.97     59291
         1.0       0.92      0.98      0.95     35506

    accuracy                           0.96     94797
   macro avg       0.95      0.97      0.96     94797
weighted avg       0.96      0.96      0.96     94797

Accuracy: 96.160216040592
